In [2]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, RegexParser
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import json
import torch
import re
import time

from knowledgegraph import KnowledgeGraph

In [3]:
with open("hf.key") as f:
    hf_token = f.read()

In [4]:
BYTES_IN_GB = 1000_000_000

def print_mem(msg = ""):
    (free, total) = torch.cuda.mem_get_info()
    used = total - free
    
    perc_usaged = round(used / total * 100.0, 1)
    used_gb = round(used / BYTES_IN_GB, 1)
    total_gb = round(total / BYTES_IN_GB, 1)
    print(f'CUDA mem usage: {used_gb}/{total_gb}GB ({perc_usaged}%)')

print_mem()

CUDA mem usage: 2.6/12.5GB (21.0%)


In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
MODEL_NAME= "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    token=hf_token
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=hf_token
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2048,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
context = """
This is written in the train from Varna to Galatz. Last
night we all assembled a little before the time of sunset. Each of us
had done his work as well as he could; so far as thought, and endeavour,
and opportunity go, we are prepared for the whole of our journey, and
for our work when we get to Galatz. When the usual time came round Mrs.
Harker prepared herself for her hypnotic effort; and after a longer and
more serious effort on the part of Van Helsing than has been usually
necessary, she sank into the trance. Usually she speaks on a hint; but
this time the Professor had to ask her questions, and to ask them pretty
resolutely, before we could learn anything; at last her answer came:--
“I can see nothing; we are still; there are no waves lapping, but only a
steady swirl of water softly running against the hawser. I can hear
men’s voices calling, near and far, and the roll and creak of oars in
the rowlocks. A gun is fired somewhere; the echo of it seems far away.
There is tramping of feet overhead, and ropes and chains are dragged
along. What is this? There is a gleam of light; I can feel the air
blowing upon me.”
Here she stopped. She had risen, as if impulsively, from where she lay
on the sofa, and raised both her hands, palms upwards, as if lifting a
weight. Van Helsing and I looked at each other with understanding.
Quincey raised his eyebrows slightly and looked at her intently, whilst
Harker’s hand instinctively closed round the hilt of his Kukri. There
was a long pause. We all knew that the time when she could speak was
passing; but we felt that it was useless to say anything. Suddenly she
sat up, and, as she opened her eyes, said sweetly:--
“Would none of you like a cup of tea? You must all be so tired!” We
could only make her happy, and so acquiesced. She bustled off to get
tea; when she had gone Van Helsing said:--
“You see, my friends. _He_ is close to land: he has left his
earth-chest. But he has yet to get on shore. In the night he may lie
hidden somewhere; but if he be not carried on shore, or if the ship do
not touch it, he cannot achieve the land. In such case he can, if it be
in the night, change his form and can jump or fly on shore, as he did
at Whitby. But if the day come before he get on shore, then, unless he
be carried he cannot escape. And if he be carried, then the customs men
may discover what the box contain. Thus, in fine, if he escape not on
shore to-night, or before dawn, there will be the whole day lost to him.
We may then arrive in time; for if he escape not at night we shall come
on him in daytime, boxed up and at our mercy; for he dare not be his
true self, awake and visible, lest he be discovered.”
"""

In [11]:
class Relation(BaseModel):
    source: str = Field(description="Name")
    target: str = Field(description="Name")
    type: str = Field(description="Type of relation. E.g.: Friend, colleague, relative")
    
class Relations(BaseModel):
    relations: List[Relation]

rel_parser = PydanticOutputParser(pydantic_object=Relations)

print(rel_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Relation": {"properties": {"source": {"description": "Name", "title": "Source", "type": "string"}, "target": {"description": "Name", "title": "Target", "type": "string"}, "type": {"description": "Type of relation. E.g.: Friend, colleague, relative", "title": "Type", "type": "string"}}, "required": ["source", "target", "type"], "title": "Relation", "type": "object"}}, "properties": {"relations": {"items": {"$ref": "#/$defs/Relation"}, "title": "Relations", "type": "array"}}, "required": ["relations"]}
```


In [17]:
messages = [
    {"role": "assistant", "content": "You are an expert on reading novels. Read the given context and answer the question of the user."},
    {"role": "user", "content": "Find all people and their relationship to each other.\n\n{format_instructions}\n\n{context}\n\n"}
]

template = tokenizer.apply_chat_template(messages, tokenize=False)
relation_prompt = PromptTemplate(
    template=template,
    input_variables=["context"],
    partial_variables={"format_instructions": rel_parser.get_format_instructions()}
)

print(relation_prompt)

relation_chain = relation_prompt | llm.bind(skip_prompt=True)

r = relation_chain.invoke(input={"context": context})
print(r)

re.findall(r'```json(.*?)```', r, re.DOTALL)

input_variables=['context'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Relation": {"properties": {"source": {"description": "Name", "title": "Source", "type": "string"}, "target": {"description": "Name", "title": "Target", "type": "string"}, "type": {"description": "Type of relation. E.g.: Friend, colleague, relative", "title": "Type", "type": "string"}}, "required": ["source", "target", "type"], "title": "Relation", "type": "object"}}, "properties": {"relations": {"items": {"$ref": "#/$defs/Relation"}, "title

['\n{\n  "relations": [\n    {\n      "source": "Van Helsing",\n      "target": "Mrs. Harker",\n      "type": "Traveler"\n    },\n    {\n      "source": "Van Helsing",\n      "target": "Quincey",\n      "type": "Traveler"\n    },\n    {\n      "source": "Van Helsing",\n      "target": "Professor",\n      "type": "Traveler"\n    },\n    {\n      "source": "Mrs. Harker",\n      "target": "Quincey",\n      "type": "Traveler"\n    },\n    {\n      "source": "Mrs. Harker",\n      "target": "Professor",\n      "type": "Traveler"\n    },\n    {\n      "source": "Quincey",\n      "target": "Professor",\n      "type": "Traveler"\n    }\n  ]\n}\n']

In [23]:
class KnowledgeGraphLLM:
    def __init__(self):
        self.kg = KnowledgeGraph()
        self.kg.clear()
        self.llm = llm
        
        self.relation_chain = relation_chain

    def extract_relationships(self, context: str) -> List[Dict[str, str]]:
        output = self.relation_chain.invoke(input={"context": context})
        relationships = []
        
        try:
            results = re.findall(r'```json(.*?)```', output, re.DOTALL)
            relationships = rel_parser.parse(results)
            
            if results:
                data = json.loads(results[-1])
                relationships = data.get("relations", [])

                for relation in relationships:
                    source = relation["source"]
                    target = relation["target"]
                    relationship = relation["type"]
                    
                    self.kg.add_node(source, {"name": source})
                    self.kg.add_node(target, {"name": target})
                    self.kg.add_edge(source, target, relationship)

        except Exception as e:
            print("Error: Could not parse LLM response as JSON\n", e)
            return []

        return relationships
    
    def graph_data(self):
        return self.kg.dump()
        
    def graph_summary(self) -> Dict[str, Any]:
        return {
            "node_count": len(self.kg.nodes),
            "edge_count": sum(len(edges) for edges in self.kg.edges.values()),
            "data": self.kg.dump()
        }

# Example usage
def query_llm(kg_llm, contexts, questions):
    # Add information through unstructured text
    # Extract and add relationships

    for i, ctx in enumerate(contexts):
        t0 = time.time()

        relationships = kg_llm.extract_relationships(ctx)

        t1 = time.time()
        print(f'Read chunk {i} in {round(t1-t0, 1)} seconds, {round(i / len(contexts) * 100.0, 1)}% done.')
        if i % 5 == 0:
            print(kg_llm.graph_data())
            torch.cuda.empty_cache()
            print_mem()            

    
    # Query the enhanced knowledge graph
    for question in questions:
        answer = kg_llm.smart_query(question)
        print(f"###\n\nQ: {question}\nA: {answer}\n\n###")
    
    # Get graph summary
    print("\nKnowledge Graph Summary:")
    print(json.dumps(kg_llm.graph_summary(), indent=2))

    return kg_llm

In [ ]:
def parse_book():
    # Initialize the system
    kg_llm = KnowledgeGraphLLM()
    
    with open("data/dracula.txt") as f:
        context = f.read()
        context = context.rpartition(r"\n\nDRACULA\n\n")[-1]

    paragraphs = []
    chapters = re.split(r"\n\nCHAPTER\ .*\n", context)
    for c in chapters[:5]:
        for p in re.split(r"\n\n", c):
            paragraphs.append(p)

    questions = [
        # "Who are the main characters in this story?",
        # "What is their relation to each other?",
        # "Who is the antagonist of the story?",
        # "How does the antagonist end up?"
    ]
    
    query_llm(kg_llm, paragraphs questions)

In [ ]:
parse_book()